## 欢迎进入 Notebook  

这里你可以编写代码，文档  

### 关于文件目录  


**project**：project 目录是本项目的工作空间，可以把将项目运行有关的所有文件放在这里，目录中文件的增、删、改操作都会被保留  


**input**：input 目录是数据集的挂载位置，所有挂载进项目的数据集都在这里，未挂载数据集时 input 目录被隐藏  


**temp**：temp 目录是临时磁盘空间，训练或分析过程中产生的不必要文件可以存放在这里，目录中的文件不会保存  


In [24]:
# 所有使用到的库
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request
import numpy as np
from sklearn.impute import KNNImputer
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [25]:
# 查看个人持久化工作区文件
!ls /home/mw/project/

ruc_Class25Q1_community.csv  ruc_Class25Q1_xtest.csv   ruc_Class25Q1_ytrain.csv
ruc_Class25Q1_merged.csv     ruc_Class25Q1_xtrain.csv
ruc_Class25Q1_train1.csv     ruc_Class25Q1_ytest.csv


In [26]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

quant4533


In [27]:
prediction = pd.read_csv('/home/mw/input/quant4533/ruc_Class25Q1_test.csv')  

# 同步处理

## 房屋房型 

In [28]:
# 定义提取函数
def extract_numbers(layout):
    if not isinstance(layout, str):  # 处理 NaN 或 None
        layout = ""

    # 使用正则表达式提取数字
    match = re.findall(r'(\d+)室|(\d+)厅|(\d+)厨|(\d+)卫', layout)
    
    # 合并所有找到的数字
    numbers = [int(num) if num else 0 for group in match for num in group if num]

    # 确保返回的列表长度为 4
    while len(numbers) < 4:
        numbers.append(0)

    return numbers

# 使用 apply() 和 pd.Series 强制确保每行数据有 4 个数值
prediction[['卧室', '客厅', '厨房', '卫生间']] = prediction['房屋户型'].apply(lambda x: pd.Series(extract_numbers(x), index=['卧室', '客厅', '厨房', '卫生间']))

# 删除原始户型列
prediction.drop(columns=['房屋户型'], inplace=True)


## 所在楼层

In [29]:
# 提取括号前的文本并替换原数据
prediction['所在楼层'] = prediction['所在楼层'].str.extract(r'(.+?)（')
# 使用 pd.get_dummies进行独热编码
prediction = pd.get_dummies(prediction, columns=['所在楼层'])

## 房屋朝向

In [30]:
# 按主要采光面方向分类（用每个数据的第一个字替换整个字符串）
prediction['房屋朝向'] = prediction['房屋朝向'].astype(str).str[0]
prediction['房屋朝向'] = prediction['房屋朝向'].fillna('其他')
# 创建一个映射字典
order_mapping = {
    '南': 0,
    '北': 1,
    '西': 2,
    '东': 3
}

# 使用映射字典进行编码
prediction['房屋朝向'] = prediction['房屋朝向'].map(order_mapping)


## 建筑结构

In [31]:
# 进行频数编码
prediction['建筑结构'] = prediction['建筑结构'].map(prediction['建筑结构'].value_counts())

## 装修情况

In [32]:
prediction['装修情况'] = prediction['装修情况'].fillna('其他')
# 创建一个映射字典
order_mapping = {
    '其他': 0,
    '毛坯': 1,
    '简装': 2,
    '精装': 3
}

# 使用映射字典进行编码
prediction['装修情况'] = prediction['装修情况'].map(order_mapping)

## 梯户比例

In [33]:
# 中文数字转换为阿拉伯数字的映射
num_map = {'一': 1, '二': 2, '两': 2, '三': 3, '四': 4, '五': 5, '六': 6, 
           '七': 7, '八': 8, '九': 9, '十': 10}

# 处理中文数字（包括大于10的情况）
def chinese_to_number(chinese_num):
    if not chinese_num:  # 处理空值
        return None
    if chinese_num in num_map:
        return num_map[chinese_num]  # 直接映射
    elif '十' in chinese_num:
        parts = chinese_num.split('十')
        if parts[0] == '':  # "十X" 形式，如"十五"
            return 10 + num_map.get(parts[1], 0)
        elif parts[1] == '':  # "X十" 形式，如"二十"
            return num_map[parts[0]] * 10
        else:  # "X十Y" 形式，如"二十五"
            return num_map[parts[0]] * 10 + num_map.get(parts[1], 0)
    return None  # 无法识别的情况返回 None

# 解析 "X梯Y户" 并计算梯户比
def convert_ratio(text):
    text = str(text)  # 确保 text 为字符串
    match = re.findall(r'([\u4e00-\u9fa5]+)梯([\u4e00-\u9fa5]+)户', text)
    if match:
        ladders = chinese_to_number(match[0][0])  # 提取电梯数
        households = chinese_to_number(match[0][1])  # 提取户数
        if ladders and households:
            return round(ladders / households, 4)  # 计算梯户比
    return None  # 无法解析的情况

# 应用转换
prediction['梯户比例'] = prediction['梯户比例'].apply(convert_ratio)

## 配备电梯

In [34]:
prediction['配备电梯'] = prediction['配备电梯'].fillna('无')
# 创建一个映射字典
order_mapping = {
    '无': 0,
    '有': 1,
}

# 使用映射字典进行编码
prediction['配备电梯'] = prediction['配备电梯'].map(order_mapping)

## 别墅类型

In [35]:
prediction['别墅类型'] = prediction['别墅类型'].fillna('非别墅')
# 使用 pd.get_dummies进行独热编码
prediction = pd.get_dummies(prediction, columns=['别墅类型'])

## 交易时间

In [36]:
# 先确保转换为 datetime 类型
prediction['交易时间'] = pd.to_datetime(prediction['交易时间'],errors = 'coerce')

# 计算数值化的年份
prediction['交易时间'] = prediction['交易时间'].dt.year + (prediction['交易时间'].dt.dayofyear / 365)


## 交易权属

In [37]:
# 使用 pd.get_dummies进行独热编码
prediction = pd.get_dummies(prediction, columns=['交易权属'])

## 上次交易

In [38]:
# 先确保转换为 datetime 类型
prediction['上次交易'] = pd.to_datetime(prediction['上次交易'],errors = 'coerce')

# 计算数值化的年份
prediction['上次交易'] = prediction['上次交易'].dt.year + (prediction['上次交易'].dt.dayofyear / 365)

## 房屋用途

In [39]:
# 进行频数编码
prediction['房屋用途'] = prediction['房屋用途'].map(prediction['房屋用途'].value_counts())
# 查看结果
print(prediction[['房屋用途']])

        房屋用途
0      14486
1      14486
2      14486
3      14486
4      14486
...      ...
14781  14486
14782  14486
14783  14486
14784  14486
14785  14486

[14786 rows x 1 columns]


## 房屋年限

In [40]:
prediction['房屋年限'] = prediction['房屋年限'].fillna('未知')

# 创建一个映射字典
order_mapping = {
    '满五年': 5,
    '满两年': 2,
    '未满两年': 1,
    '未知':3.5 
}

# 使用映射字典进行编码
prediction['房屋年限'] = prediction['房屋年限'].map(order_mapping)

## 产权所属

In [41]:
# 创建一个映射字典
order_mapping = {
    '共有': 0,
    '非共有': 1,
}

# 使用映射字典进行编码
prediction['产权所属'] = prediction['产权所属'].map(order_mapping)


## 房屋优势

In [42]:
prediction.drop(columns=['房屋优势'], inplace=True)


In [43]:
def label(column_name):
    # 非空值为1，空值为0
    prediction[column_name] = prediction[column_name].isna().astype(int)
    return prediction
for i in ['核心卖点','户型介绍','周边配套','交通出行']:
    label(i)

# 合并and填充

In [44]:
community = pd.read_csv('/home/mw/project/ruc_Class25Q1_community.csv') 

In [45]:
# 合并小区数据到房价数据
mtest = prediction.merge(community, on='小区名称', how='left')
print(mtest)

          ID  城市    区域     板块    环线       小区名称     建筑面积     套内面积  房屋朝向  \
0          0   0  45.0  416.0  四至五环       泛海容郡   209.2㎡  165.71㎡     0   
1          1   0  45.0  414.0  四至五环     慧谷金色家园  163.69㎡      NaN     0   
2          2   0  43.0  289.0  五至六环      天通苑中苑  102.92㎡      NaN     2   
3          3   0  39.0  374.0   NaN     富乐小区南里  109.66㎡      NaN     0   
4          4   0  79.0  724.0  二至三环      百万庄未区    57.2㎡      NaN     0   
...      ...  ..   ...    ...   ...        ...      ...      ...   ...   
16105  14781   6  71.0  244.0   NaN  横巷市委家属院南区   65.78㎡      NaN     0   
16106  14782   6  91.0  723.0   NaN       润天小区  138.01㎡      NaN     0   
16107  14783   6  83.0  380.0   三环外       御湖蓝湾     110㎡      NaN     0   
16108  14784   6  83.0  380.0   三环外       御湖蓝湾     136㎡      NaN     0   
16109  14785   6  91.0  723.0   NaN       润天小区     183㎡      NaN     0   

          建筑结构  ...  绿 化 率  容 积 率  物 业 费     供水        供暖     供电   燃气费   供热费  \
0      10794.0  ...     35   2.

In [46]:
mtest['环线'] = mtest['环线'].fillna(mtest['环线位置'])
mtest.drop(columns = ['环线位置'], inplace = True)
mtest.drop(columns = ['小区名称'], inplace = True)

In [47]:
most_frequent_value = mtest['供暖'].mode()[0]
mtest['供暖'] = mtest['供暖'].fillna(most_frequent_value)
# 供暖方式类别
heating_types = ['集中供暖', '自采暖', '无供暖']

# 创建独热编码列
for heating in heating_types:
    mtest[heating] = mtest['供暖'].apply(lambda x: 1 if heating in x else 0)

mtest.drop(columns = ['供暖'], inplace = True)

In [48]:
most_frequent_value = mtest['供水'].mode()[0]
mtest['供水'] = mtest['供水'].fillna(most_frequent_value)
waterSupply_types = ['民水','商水']

# 创建独热编码列
for waterSupply in waterSupply_types:
    mtest[waterSupply] = mtest['供水'].apply(lambda x: 1 if waterSupply in x else 0)

mtest.drop(columns = ['供水'], inplace = True)

In [49]:
most_frequent_value = mtest['供电'].mode()[0]
mtest['供电'] = mtest['供电'].fillna(most_frequent_value)
elecSupply_types = ['民电','商电']

# 创建独热编码列
for elecSupply in elecSupply_types:
    mtest[elecSupply] = mtest['供电'].apply(lambda x: 1 if elecSupply in x else 0)

mtest.drop(columns = ['供电'], inplace = True)

In [50]:
mapping = {
    "一环内": 1,
    "内环内": 2,
    "二环内": 3,
    "一至二环": 1.5,
    "二至三环": 2.5,
    "三至四环": 3.5,
    "四至五环": 4.5,
    "五至六环": 5.5,
    "六环外": 6,
    "三环外": 4.5,
    "四环外": 5,
    "外环外": 7,
    "内环至中环": 3,
    "中环至外环": 5,
    "内环至外环": 4
}

mtest["环线"] = mtest["环线"].replace(mapping)

In [51]:
mtest['绿 化 率'] = pd.to_numeric(mtest['绿 化 率'], errors='coerce')
mtest.drop(columns = ["抵押信息"], inplace = True)

In [52]:
# 确保所有数据都转换为字符串类型后去除平方米单位
mtest['建筑面积'] = mtest['建筑面积'].astype(str).str.replace('㎡', '', regex=True).astype(float)
mtest['套内面积'] = mtest['套内面积'].astype(str).str.replace('㎡', '', regex=True)
mtest['套内面积'] = pd.to_numeric(mtest['套内面积'], errors='coerce')  # 转换为数值，无法转换的变为 NaN

In [53]:
# 按建筑面积降序排序
mtest = mtest.sort_values(by='建筑面积', ascending=False).reset_index(drop=True)

# 使用建筑面积作为参考，但只填充套内面积
temp_mtest = mtest[['建筑面积', '套内面积']]
imputer = KNNImputer(n_neighbors=10)
mtest['套内面积'] = imputer.fit_transform(temp_mtest)[:, 1]  # 仅填充套内面积

# 确保填充后的套内面积不会超过建筑面积
mtest['套内面积'] = np.minimum(mtest['套内面积'], mtest['建筑面积'])

# 计算得房率（套内面积 / 建筑面积）
mtest['得房率'] = mtest['套内面积'] / mtest['建筑面积']

In [54]:
for col in mtest.iloc[:, 0:].columns:  # 选取从第4列（索引3）往后的所有列
    mtest[col] = pd.to_numeric(mtest[col], errors='coerce')  # 转换为数值，错误变 NaN
    if mtest[col].notna().sum() > 0:  # 仅在列中有有效数字时才计算众数
        mode_value = mtest[col].mode()[0]  # 获取众数
        mtest[col].fillna(mode_value, inplace=True)  # 用众数填充 NaN

print(mtest)

          ID  城市    区域     板块   环线    建筑面积     套内面积  房屋朝向     建筑结构  装修情况  ...  \
0       7774   3  49.0  491.0  7.0  594.38  311.689     0  10794.0     1  ...   
1       1995   0  43.0  488.0  5.5  568.91  311.689     0  10794.0     1  ...   
2        500   0  43.0  488.0  5.5  565.35  311.689     0  10794.0     1  ...   
3       7237   3  25.0  600.0  7.0  515.82  311.689     0  10794.0     2  ...   
4       2271   0  62.0  655.0  5.5  468.57  311.689     0  10794.0     3  ...   
...      ...  ..   ...    ...  ...     ...      ...   ...      ...   ...  ...   
16105   5164   2  57.0  188.0  7.0   21.00   21.000     3   1330.0     0  ...   
16106   5130   2  57.0  188.0  7.0   21.00   21.000     0   1330.0     0  ...   
16107   6347   2  57.0  188.0  7.0   20.78   20.780     0   1330.0     0  ...   
16108   9942   4   3.0  466.0  2.5   19.53   18.390     0  10794.0     2  ...   
16109  12060   4  70.0  192.0  2.5   18.76   18.760     1  10794.0     0  ...   

          停车位   停车费用  集中供暖 

In [55]:
mtest.to_csv('/home/mw/project/ruc_Class25Q1_predictiontest.csv', index=False)

In [56]:
mtest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16110 entries, 0 to 16109
Data columns (total 66 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            16110 non-null  int64  
 1   城市            16110 non-null  int64  
 2   区域            16110 non-null  float64
 3   板块            16110 non-null  float64
 4   环线            16110 non-null  float64
 5   建筑面积          16110 non-null  float64
 6   套内面积          16110 non-null  float64
 7   房屋朝向          16110 non-null  int64  
 8   建筑结构          16110 non-null  float64
 9   装修情况          16110 non-null  int64  
 10  梯户比例          16110 non-null  float64
 11  配备电梯          16110 non-null  int64  
 12  交易时间          16110 non-null  float64
 13  上次交易          16110 non-null  float64
 14  房屋用途          16110 non-null  int64  
 15  房屋年限          16110 non-null  float64
 16  产权所属          16110 non-null  int64  
 17  核心卖点          16110 non-null  int64  
 18  户型介绍          16110 non-nu